In [3]:
##################################################################
################### last update 20230228 #########################
##################################################################
def add_arrow_peachpuff(
    axes, line, arrow_locs=[0.2, 0.4, 0.6, 0.8],
    arrowstyle='-|>', arrowsize=1, transform=None):
    if not isinstance(line, mlines.Line2D):
        raise ValueError("expected a matplotlib.lines.Line2D object")
    x, y = line.get_xdata(), line.get_ydata()

    arrow_kw = {
        "arrowstyle": arrowstyle,
        "mutation_scale": 10 * arrowsize,
    }

    color = 'peachpuff'
    #line.get_color()
    use_multicolor_lines = isinstance(color, np.ndarray)
    if use_multicolor_lines:
        raise NotImplementedError("multicolor lines not supported")
    else:
        arrow_kw['color'] = color

    linewidth = line.get_linewidth()
    if isinstance(linewidth, np.ndarray):
        raise NotImplementedError("multiwidth lines not supported")
    else:
        arrow_kw['linewidth'] = linewidth

    if transform is None:
        transform = axes.transData

    arrows = []
    for loc in arrow_locs:
        s = np.cumsum(np.sqrt(np.diff(x) ** 2 + np.diff(y) ** 2))
        n = np.searchsorted(s, s[-1] * loc)
        arrow_tail = (x[n], y[n])
        arrow_head = (np.mean(x[n:n + 2]), np.mean(y[n:n + 2]))
        p = mpatches.FancyArrowPatch(
            arrow_tail, arrow_head, transform=transform,
            **arrow_kw)
        axes.add_patch(p)
        arrows.append(p)
    return arrows

def time_1D(x, number_of_raters):
    size=[500/number_of_raters]
    for i in range(len(x)-1):
        if x[i]==x[i+1]:
            size.append(size[i]+500/number_of_raters)
        else:
            size.append(500/number_of_raters)
    return size



#---------------------- 1D ASSG excluding global-----------------------
def plot_1D_ASSG(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(2*(5.4-0.6),2*(number_of_raters+2)*1.1))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
#y-fit for expected values------------------------------------------------------------
    popty, pcovy = curve_fit(xconst,v[j], ratermeanvaluelist(v))
    #yfit_yconst=np.linspace(0, number_of_raters+0.5)
    #xfit_yconst=popty[0]+0*yfit_yconst
    #ax.plot(xfit_yconst, yfit_yconst, color='crimson')
        
#mean value of all ratings------------------------------------------------------------
    mean_ratings, error_ratings = ratings_mean_and_error(all_ratings, number_of_raters, number_of_ratings)
    const_mean_values=np.zeros(len(mean_ratings))+number_of_raters+1
    mean_ratings_reloc, mean_ratings_reloc_row = relocating(mean_ratings, const_mean_values)
    mean_values_plot_arrows= ax.plot(mean_ratings_reloc, mean_ratings_reloc_row,
                                 c='peachpuff', zorder=0)[0]
    mean_values_plot= ax.scatter(mean_ratings_reloc, mean_ratings_reloc_row, s=time_1D(mean_ratings,number_of_raters),
                                 c=df.z, cmap='RdPu', zorder=10)
    mean_values_plot= ax.scatter(mean_ratings_reloc[0], mean_ratings_reloc_row[0], s=time_1D(mean_ratings,number_of_raters)[0],
                                 c='hotpink', 
                                 label='mean values of raters \n$\overline{\Delta rating}=$%.2f$\pm$%.2f'
                                 %(meanlist(error_ratings), stdlist(error_ratings)),
                                 zorder=0)
#color for meanvalues between ratings
    color_lines = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
    points=np.array([mean_ratings_reloc, mean_ratings_reloc_row]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = mc.LineCollection(segments, cmap='RdPu', norm=norm)
    ax.add_collection(lc)
    lc.set_array(np.linspace(10, len(mean_ratings_reloc), len(mean_ratings_reloc)))
                
    add_arrow_hotpink(ax, mean_values_plot_arrows, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
    #EV
    popty, pcovy = curve_fit(xconst,range(1, number_of_raters), ratermeanvaluelist(v))
    ax.scatter(popty[0], number_of_raters+1, s=400, color='crimson',
                   label='$\overline{EV}$=%.2f$\pm$%.2f'%(popty[0], stdlist(ev_list)/np.sqrt(number_of_raters)), zorder=20)
    #start and end
    ax.scatter(mean_ratings_reloc[0],mean_ratings_reloc_row[0] ,s=50, color='limegreen', zorder=25)
    ax.scatter(mean_ratings_reloc[-1],mean_ratings_reloc_row[-1] ,s=20, color='gold', zorder=30)
    stdx=(np.linspace(meanlist(mean_ratings)-stdlist(mean_ratings),
                    meanlist(mean_ratings)+stdlist(mean_ratings), 50))
    #SD
    stdx=np.linspace(popty[0]-meanlist(meanvalue_sd), popty[0]+meanlist(meanvalue_sd), 50)
    stdy=stdx*0+len(v)+1
    ax.plot(stdx, stdy, linewidth='3', color='orangered', 
                label='$\overline{SD}$=%.2f$\pm$%.2f'%(mean_sd, stdlist(meanvalue_sd)/np.sqrt(number_of_raters)), zorder=15)
    #color errorbars
    errorbar_color = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    cmap = cm.RdPu
    norm = Normalize(vmin=errorbar_color.min(), vmax=errorbar_color.max())
    plt.errorbar(mean_ratings_reloc, mean_ratings_reloc_row, 0.25*error_ratings, error_ratings,
                 fmt='none', ecolor='whitesmoke', elinewidth=3, zorder=10)
    plt.errorbar(mean_ratings_reloc, mean_ratings_reloc_row, 0.25*error_ratings, error_ratings, fmt='none',
                 ecolor=cmap(norm(errorbar_color)), elinewidth=7, zorder=8)
     
    
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+2)*1.1])
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel(
            ('Rating, $\overline{td}$=%.2f$\pm$%.2f,'
             '$\overline{tt}=%.2f$$\pm$%.2f')
                      %(round(meanlist(travel_distance_list), 2),
                        round(stdlist(travel_distance_list)/np.sqrt(number_of_raters), 2),
                        round(meanlist(travel_tendency_list), 2),
                        round(stdlist(travel_tendency_list)/np.sqrt(number_of_raters), 2)),
             fontsize='28')
    ax.set_ylabel('Rater', fontsize='24')
###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='17').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   

    
#---------------------- 1D ASSG including global-----------------------
def plot_1D_ASSG_including_global(path_data, path_data_global,path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    global_not_arranged=np.loadtxt(path_data_global, unpack=True)
    global_all=np.array(v_not_arranged).T
    global_single_indicator=global_all[q]
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(number_of_raters*1.4)))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    #ax.scatter(global_single_indicator[0], const[0], s=100, color='fuchsia', label='global rating', zorder=21)
#y-fit for expected values------------------------------------------------------------
    popty, pcovy = curve_fit(xconst,range(1, number_of_raters), ratermeanvaluelist(v))
    yfit_yconst=np.linspace(0, number_of_raters+1.5)
    xfit_yconst=popty[0]+0*yfit_yconst
    ax.plot(xfit_yconst, yfit_yconst, color='crimson', zorder=10)
#y-fit for global values------------------------------------------------------------
    xfit_yconst=meanlist(global_single_indicator)+0*yfit_yconst
    ax.plot(xfit_yconst, yfit_yconst, color='fuchsia', zorder=10)
#mean value of all ratings------------------------------------------------------------
    mean_ratings, error_ratings = ratings_mean_and_error(all_ratings, number_of_raters, number_of_ratings)
    const_mean_values=np.zeros(len(mean_ratings))+number_of_raters+1
    mean_ratings_reloc, mean_ratings_reloc_row = relocating(mean_ratings, const_mean_values)
    mean_values_plot_arrows= ax.plot(mean_ratings_reloc, mean_ratings_reloc_row,
                                 c='peachpuff', zorder=0)[0]
    mean_values_plot= ax.scatter(mean_ratings_reloc, mean_ratings_reloc_row, s=time_1D(mean_ratings,number_of_raters),
                                 c=df.z, cmap='RdPu', zorder=10)
    mean_values_plot= ax.scatter(mean_ratings_reloc[0], mean_ratings_reloc_row[0], s=time_1D(mean_ratings,number_of_raters)[0],
                                 c='hotpink', 
                                 label='mean values of raters \n$\overline{\Delta rating}=$%.2f$\pm$%.2f'
                                 %(meanlist(error_ratings), stdlist(error_ratings)),
                                 zorder=0)
#color for meanvalues between ratings
    color_lines = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
    points=np.array([mean_ratings_reloc, mean_ratings_reloc_row]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = mc.LineCollection(segments, cmap='RdPu', norm=norm)
    ax.add_collection(lc)
    lc.set_array(np.linspace(10, len(mean_ratings_reloc), len(mean_ratings_reloc)))
                
    add_arrow_hotpink(ax, mean_values_plot_arrows, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
    #EV
    ax.scatter(popty[0], number_of_raters+1, s=400, color='crimson',
                   label='$\overline{EV}$=%.2f$\pm$%.2f'%(popty[0], stdlist(ev_list)/np.sqrt(number_of_raters)), zorder=20)
    #start and end
    ax.scatter(mean_ratings_reloc[0],mean_ratings_reloc_row[0] ,s=50, color='limegreen', zorder=25)
    ax.scatter(mean_ratings_reloc[-1],mean_ratings_reloc_row[-1] ,s=20, color='gold', zorder=30)
    stdx=(np.linspace(meanlist(mean_ratings)-stdlist(mean_ratings),
                    meanlist(mean_ratings)+stdlist(mean_ratings), 50))
    #SD
    stdx=np.linspace(popty[0]-meanlist(meanvalue_sd), popty[0]+meanlist(meanvalue_sd), 50)
    stdy=stdx*0+len(v)+1
    ax.plot(stdx, stdy, linewidth='3', color='orangered', 
                label='$\overline{SD}$=%.2f$\pm$%.2f'%(mean_sd, stdlist(meanvalue_sd)/np.sqrt(number_of_raters)), zorder=15)
    #global    
    ax.scatter(meanlist(global_single_indicator), const_mean_values[0], s=400, color='fuchsia',
               label='global ratings \n$|\overline{global}-\overline{EV}|$=%.2f'
               %(np.sqrt((popty[0]-meanlist(global_single_indicator))**2)),
               zorder=21)
    #color errorbars
    errorbar_color = df.z
    cmap = cm.RdPu
    norm = Normalize(vmin=errorbar_color.min(), vmax=errorbar_color.max())
    plt.errorbar(mean_ratings_reloc, mean_ratings_reloc_row, error_ratings, error_ratings,
                 fmt='none', ecolor='whitesmoke', elinewidth=3, zorder=10)
    plt.errorbar(mean_ratings_reloc, mean_ratings_reloc_row, error_ratings, error_ratings, fmt='none',
                 ecolor=cmap(norm(df.z)), elinewidth=7, zorder=8)
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+1)*1.4])
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel(
            ('Rating, $\overline{td}$=%.2f$\pm$%.2f,'
             '$\overline{tt}=%.2f$$\pm$%.2f')
                      %(round(meanlist(travel_distance_list), 2),
                        round(stdlist(travel_distance_list)/np.sqrt(number_of_raters), 2),
                        round(meanlist(travel_tendency_list), 2),
                        round(stdlist(travel_tendency_list)/np.sqrt(number_of_raters), 2)),
             fontsize='28')
    ax.set_ylabel('Rater', fontsize='24')

###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   


In [9]:
def plot_1D_ASSG_including_pre_and_post_global(path_data, path_pre_global, path_post_global, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v_not_sliced=np.array(v_not_arranged).T
    pre_global_not_arranged=np.loadtxt(path_pre_global, unpack=True)
    pre_global=np.array(pre_global_not_arranged).T
    post_global_not_arranged=np.loadtxt(path_post_global, unpack=True)
    post_global=np.array(post_global_not_arranged).T
    v=v_not_sliced[:len(pre_global)]
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(2*(5.4-0.6),2*(number_of_raters+2)*1.1))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #scatter for pre and post global
                ax.scatter(pre_global[j],j+1 ,s=2000/number_of_raters, color='crimson', marker=6, zorder=35)
                ax.scatter(post_global[j],j+1 ,s=2000/number_of_raters, color='crimson', marker=7, zorder=40)
                
                
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
#y-fit for expected values------------------------------------------------------------
    popty, pcovy = curve_fit(xconst,v[j], ratermeanvaluelist(v))
    yfit_yconst=np.linspace(0, number_of_raters+1.5)
    xfit_yconst=popty[0]+0*yfit_yconst
    ax.plot(xfit_yconst, yfit_yconst, color='crimson', ls='solid')
    
#y-fit for post global values------------------------------------------------------------
    yfit_yconst_pre_global=np.linspace(0, number_of_raters+1.5)
    xfit_yconst_pre_global=meanlist(pre_global)+0*yfit_yconst_pre_global
    ax.plot(xfit_yconst_pre_global, yfit_yconst_pre_global, color='crimson', ls='-.', zorder=10)
    
#y-fit for post global values------------------------------------------------------------
    yfit_yconst_post_global=np.linspace(0, number_of_raters+1.5)
    xfit_yconst_post_global=meanlist(post_global)+0*yfit_yconst_pre_global
    ax.plot(xfit_yconst_post_global, yfit_yconst_post_global, color='crimson', ls='dotted', zorder=10)
    
#mean value of all ratings------------------------------------------------------------
    mean_ratings, error_ratings = ratings_mean_and_error(all_ratings, number_of_raters, number_of_ratings)
    const_mean_values=np.zeros(len(mean_ratings))+number_of_raters+1
    mean_ratings_reloc, mean_ratings_reloc_row = relocating(mean_ratings, const_mean_values)
    mean_values_plot_arrows= ax.plot(mean_ratings_reloc, mean_ratings_reloc_row,
                                 c='peachpuff', zorder=0)[0]
    mean_values_plot= ax.scatter(mean_ratings_reloc, mean_ratings_reloc_row, s=time_1D(mean_ratings,number_of_raters),
                                 c=df.z, cmap='RdPu', zorder=10)
    mean_values_plot= ax.scatter(mean_ratings_reloc[0], mean_ratings_reloc_row[0], s=time_1D(mean_ratings,number_of_raters)[0],
                                 c='hotpink', 
                                 label='mean values of raters \n$\overline{\Delta rating}=$%.2f$\pm$%.2f'
                                 %(meanlist(error_ratings), stdlist(error_ratings)),
                                 zorder=0)
#color for meanvalues between ratings
    color_lines = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
    points=np.array([mean_ratings_reloc, mean_ratings_reloc_row]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = mc.LineCollection(segments, cmap='RdPu', norm=norm)
    ax.add_collection(lc)
    lc.set_array(np.linspace(10, len(mean_ratings_reloc), len(mean_ratings_reloc)))
                
    add_arrow_hotpink(ax, mean_values_plot_arrows, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
    #EV
    popty, pcovy = curve_fit(xconst,range(1, number_of_raters), ratermeanvaluelist(v))
    ax.scatter(popty[0], number_of_raters+1, s=400, color='crimson',
                   label='$\overline{EV}$=%.2f$\pm$%.2f'%(popty[0], stdlist(ev_list)/np.sqrt(number_of_raters)), zorder=20)
    #globals
    
    ax.scatter(round(meanlist(post_global),2), number_of_raters+1, s=400, color='crimson', marker=7,
                   label='$\overline{post\, video\, global}$=%s$\pm$%s'
               %(round(meanlist(post_global),2), round(stdlist(post_global)/np.sqrt(len(pre_global)),2)), zorder=40)
    ax.scatter(round(meanlist(pre_global),2), number_of_raters+1, s=400, color='crimson', marker=6,
                   label='$\overline{pre\, video\, global}$=%s$\pm$%s'
               %(round(meanlist(pre_global),2), round(stdlist(pre_global)/np.sqrt(len(pre_global)),2)), zorder=35)
    
    
    #start and end
    ax.scatter(mean_ratings_reloc[0],mean_ratings_reloc_row[0] ,s=50, color='limegreen', zorder=25)
    ax.scatter(mean_ratings_reloc[-1],mean_ratings_reloc_row[-1] ,s=20, color='gold', zorder=30)
    stdx=(np.linspace(meanlist(mean_ratings)-stdlist(mean_ratings),
                    meanlist(mean_ratings)+stdlist(mean_ratings), 50))
    #SD
    stdx=np.linspace(popty[0]-meanlist(meanvalue_sd), popty[0]+meanlist(meanvalue_sd), 50)
    stdy=stdx*0+len(v)+1
    ax.plot(stdx, stdy, linewidth='3', color='orangered', 
                label='$\overline{SD}$=%.2f$\pm$%.2f'%(mean_sd, stdlist(meanvalue_sd)/np.sqrt(number_of_raters)), zorder=15)
    #color errorbars
    errorbar_color = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    cmap = cm.RdPu
    norm = Normalize(vmin=errorbar_color.min(), vmax=errorbar_color.max())
    plt.errorbar(mean_ratings_reloc, mean_ratings_reloc_row, 0.25*error_ratings, error_ratings,
                 fmt='none', ecolor='whitesmoke', elinewidth=3, zorder=10)
    plt.errorbar(mean_ratings_reloc, mean_ratings_reloc_row, 0.25*error_ratings, error_ratings, fmt='none',
                 ecolor=cmap(norm(errorbar_color)), elinewidth=7, zorder=8)
     
    
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+2)*1.1])
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel(
            ('Rating, $\overline{td}$=%.2f$\pm$%.2f,'
             '$\overline{tt}=%.2f$$\pm$%.2f')
                      %(round(meanlist(travel_distance_list), 2),
                        round(stdlist(travel_distance_list)/np.sqrt(number_of_raters), 2),
                        round(meanlist(travel_tendency_list), 2),
                        round(stdlist(travel_tendency_list)/np.sqrt(number_of_raters), 2)),
             fontsize='28')
    ax.set_ylabel('Rater', fontsize='24')
###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='16').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   

In [ ]:
def plot_1D_ASSG_pre_and_post_global_only(path_data, path_pre_global, path_post_global, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v_not_sliced=np.array(v_not_arranged).T
    pre_global_not_arranged=np.loadtxt(path_pre_global, unpack=True)
    pre_global=np.array(pre_global_not_arranged).T
    post_global_not_arranged=np.loadtxt(path_post_global, unpack=True)
    post_global=np.array(post_global_not_arranged).T
    v=v_not_sliced[:len(pre_global)]
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(2*(5.4-0.6),2*(number_of_raters+1)*1.1))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                
                #scatter for pre and post global
                ax.scatter(pre_global[j],j+1 ,s=2000/number_of_raters, color='crimson', marker=6, zorder=35)
                ax.scatter(post_global[j],j+1 ,s=2000/number_of_raters, color='crimson', marker=7, zorder=40)
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
        
    (ax.plot(np.linspace(meanlist(v[0])-meanlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
#y-fit for expected values------------------------------------------------------------
    popty, pcovy = curve_fit(xconst,v[j], ratermeanvaluelist(v))
    yfit_yconst=np.linspace(0, number_of_raters+1.5)
    xfit_yconst=popty[0]+0*yfit_yconst
    ax.plot(xfit_yconst, yfit_yconst, color='crimson', ls='solid')
    
#y-fit for post global values------------------------------------------------------------
    yfit_yconst_pre_global=np.linspace(0, number_of_raters+1.5)
    xfit_yconst_pre_global=meanlist(pre_global)+0*yfit_yconst_pre_global
    ax.plot(xfit_yconst_pre_global, yfit_yconst_pre_global, color='crimson', ls='-.', zorder=10)
    
#y-fit for post global values------------------------------------------------------------
    yfit_yconst_post_global=np.linspace(0, number_of_raters+1.5)
    xfit_yconst_post_global=meanlist(post_global)+0*yfit_yconst_pre_global
    ax.plot(xfit_yconst_post_global, yfit_yconst_post_global, color='crimson', ls='dotted', zorder=10)

    #EV
    popty, pcovy = curve_fit(xconst,range(1, number_of_raters), ratermeanvaluelist(v))
    ax.scatter(popty[0], number_of_raters+1, s=400, color='crimson',
                   label='$\overline{EV}$=%.2f$\pm$%.2f'%(popty[0], stdlist(ev_list)/np.sqrt(number_of_raters)), zorder=20)
    #globals
    
    ax.scatter(round(meanlist(post_global),2), number_of_raters+1, s=400, color='crimson', marker=7,
                   label='$\overline{post\, video\, global}$=%s$\pm$%s'
               %(round(meanlist(post_global),2), round(stdlist(post_global)/np.sqrt(len(pre_global)),2)), zorder=40)
    ax.scatter(round(meanlist(pre_global),2), number_of_raters+1, s=400, color='crimson', marker=6,
                   label='$\overline{pre\, video\, global}$=%s$\pm$%s'
               %(round(meanlist(pre_global),2), round(stdlist(pre_global)/np.sqrt(len(pre_global)),2)), zorder=35)
    
    
    #SD
    stdx=np.linspace(popty[0]-meanlist(meanvalue_sd), popty[0]+meanlist(meanvalue_sd), 50)
    stdy=stdx*0+len(v)+1
    ax.plot(stdx, stdy, linewidth='3', color='orangered', 
                label='$\overline{SD}$=%.2f$\pm$%.2f'%(mean_sd, stdlist(meanvalue_sd)/np.sqrt(number_of_raters)), zorder=15)
     
    
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+1)*1.1])
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel(
            ('Rating, $\overline{td}$=%.2f$\pm$%.2f,'
             '$\overline{tt}=%.2f$$\pm$%.2f')
                      %(round(meanlist(travel_distance_list), 2),
                        round(stdlist(travel_distance_list)/np.sqrt(number_of_raters), 2),
                        round(meanlist(travel_tendency_list), 2),
                        round(stdlist(travel_tendency_list)/np.sqrt(number_of_raters), 2)),
             fontsize='28')
    ax.set_ylabel('Rater', fontsize='24')
###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='16').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   

In [ ]:
#---------------------- 1D ASSG excluding global-----------------------
def plot_1D_ASSG_mean_values_only(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(2*(5.4-0.6),2*(2.5)))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
                const=np.zeros(len(v[j]))+1+j
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
        
#mean value of all ratings------------------------------------------------------------
    mean_sd=meanlist(meanvalue_sd)
    mean_ratings, error_ratings = ratings_mean_and_error(all_ratings, number_of_raters, number_of_ratings)
    const_mean_values=np.zeros(len(mean_ratings))+1
    mean_ratings_reloc, mean_ratings_reloc_row = relocating(mean_ratings, const_mean_values)
    mean_values_plot_arrows= ax.plot(mean_ratings_reloc, mean_ratings_reloc_row,
                                 c='peachpuff', zorder=0)[0]
    mean_values_plot= ax.scatter(mean_ratings_reloc, mean_ratings_reloc_row, s=time_1D(mean_ratings,number_of_raters),
                                 c=df.z, cmap='RdPu', zorder=10)
    mean_values_plot= ax.scatter(mean_ratings_reloc[0], mean_ratings_reloc_row[0], s=time_1D(mean_ratings,number_of_raters)[0],
                                 c='hotpink', 
                                 label='mean values of raters \n$\overline{\Delta rating}=$%.2f$\pm$%.2f'
                                 %(meanlist(error_ratings), stdlist(error_ratings)),
                                 zorder=0)
#color for meanvalues between ratings
    color_lines = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
    points=np.array([mean_ratings_reloc, mean_ratings_reloc_row]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = mc.LineCollection(segments, cmap='RdPu', norm=norm)
    ax.add_collection(lc)
    lc.set_array(np.linspace(10, len(mean_ratings_reloc), len(mean_ratings_reloc)))
                
    add_arrow_hotpink(ax, mean_values_plot_arrows, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
    #EV
    popty, pcovy = curve_fit(xconst,range(1, number_of_raters), ratermeanvaluelist(v))
    ax.scatter(popty[0], 1, s=400, color='crimson',
                   label='$\overline{EV}$=%.2f$\pm$%.2f'%(popty[0], stdlist(ev_list)/np.sqrt(number_of_raters)), zorder=20)
    #start and end
    ax.scatter(mean_ratings_reloc[0],mean_ratings_reloc_row[0] ,s=50, color='limegreen', zorder=25)
    ax.scatter(mean_ratings_reloc[-1],mean_ratings_reloc_row[-1] ,s=20, color='gold', zorder=30)
    stdx=(np.linspace(meanlist(mean_ratings)-stdlist(mean_ratings),
                    meanlist(mean_ratings)+stdlist(mean_ratings), 50))
    #SD
    stdx=np.linspace(popty[0]-meanlist(meanvalue_sd), popty[0]+meanlist(meanvalue_sd), 50)
    stdy=stdx*0+1
    ax.plot(stdx, stdy, linewidth='3', color='orangered', 
                label='$\overline{SD}$=%.2f$\pm$%.2f'%(mean_sd, stdlist(meanvalue_sd)/np.sqrt(number_of_raters)), zorder=15)
    #color errorbars
    errorbar_color = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    cmap = cm.RdPu
    norm = Normalize(vmin=errorbar_color.min(), vmax=errorbar_color.max())
    plt.errorbar(mean_ratings_reloc, mean_ratings_reloc_row, 0.25*error_ratings, error_ratings,
                 fmt='none', ecolor='whitesmoke', elinewidth=3, zorder=10)
    plt.errorbar(mean_ratings_reloc, mean_ratings_reloc_row, 0.25*error_ratings, error_ratings, fmt='none',
                 ecolor=cmap(norm(errorbar_color)), elinewidth=7, zorder=8)
     
    
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(2.5)])
    plt.yticks([])
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel(
            ('Rating, $\overline{td}$=%.2f$\pm$%.2f,'
             '$\overline{tt}=%.2f$$\pm$%.2f')
                      %(round(meanlist(travel_distance_list), 2),
                        round(stdlist(travel_distance_list)/np.sqrt(number_of_raters), 2),
                        round(meanlist(travel_tendency_list), 2),
                        round(stdlist(travel_tendency_list)/np.sqrt(number_of_raters), 2)),
             fontsize='28')
    ax.set_ylabel('Mean values of raters', fontsize='20')
###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='17').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   


In [ ]:
###------------------- 1D ASSG sucessive excluding global--------------------

def plot_1D_ASSG_1_single_rating_SSG_format(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(1.5)))
    grid_chessboard(6, int((1.5+3)*1.6-0.5))
    for j in range(len(v)-len(v)+1): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c='skyblue', zorder=10))
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of rater', zorder=0)
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,2.0])
    plt.yticks(list(range(1, 1)))
    #axes and title
    ax.set_title('1D ASSG, indicator: %s'%(name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')

###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   


def plot_1D_ASSG_1_single_rating(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(1.5)))
    grid_chessboard(6, int((1.5+3)*1.6-0.5))
    for j in range(len(v)-len(v)+1): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,2.0])
    plt.yticks(list(range(1, 1)))
    #axes and title
    ax.set_title('1D ASSG, indicator: %s'%(name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')

###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   


    
def plot_1D_ASSG_2_single_rating_ev(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(1.5)))
    grid_chessboard(6, int((1.5+3)*1.6-0.5))
    for j in range(len(v)-len(v)+1): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater: %.2f'%(meanlist(v[0])), zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,2.0])
    plt.yticks(list(range(1, 1)))
    #axes and title
    ax.set_title('1D ASSG, indicator: %s'%(name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')
    ###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   

def plot_1D_ASSG_2_single_rating_ev_sd(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(1.5)))
    grid_chessboard(6, int((1.5+3)*1.6-0.5))
    for j in range(len(v)-len(v)+1): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater: %.2f'%(stdlist(v[0])), zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater: %.2f'%(meanlist(v[0])), zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,2.0])
    plt.yticks(list(range(1, 1)))
    #axes and title
    ax.set_title('1D ASSG, indicator: %s'%(name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')
    ###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   
    
######################################################################################################################
######################################################################################################################
def plot_1D_ASSG_2_single_rating_ev_sd_td(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(1.5)))
    grid_chessboard(6, int((1.5+3)*1.6-0.5))
    for j in range(len(v)-len(v)+1): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of rater, td=%.2f'%(2*traveldistance(v[0], const)), zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater: %.2f'%(stdlist(v[0])), zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater: %.2f'%(meanlist(v[0])), zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,2.0])
    plt.yticks(list(range(1, 1)))
    #axes and title
    ax.set_title('1D ASSG, indicator: %s'%(name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')
    ###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   
    
######################################################################################################################
######################################################################################################################
def plot_1D_ASSG_2_single_rating_ev_sd_td_tt(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(1.5)))
    grid_chessboard(6, int((1.5+3)*1.6-0.5))
    for j in range(len(v)-len(v)+1): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of rater, \ntd=%.2f, tt=%.2f'
                      %(2*traveldistance(v[0], const), 2*traveltendency(v[j], const)), zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater: %.2f'%(stdlist(v[0])), zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater: %.2f'%(meanlist(v[0])), zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,2.0])
    plt.yticks(list(range(1, 1)))
    #axes and title
    ax.set_title('1D ASSG, indicator: %s'%(name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')
    ###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   
    
######################################################################################################################
######################################################################################################################

def plot_1D_ASSG_3_two_ratings(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(2*1.4)))
    grid_chessboard(6, int((2+3)*1.6-0.5))
    for j in range(len(v)-len(v)+2): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,3.0])
    plt.yticks(list(range(1, 2)))
    #axes and title
    ax.set_title('1D ASSG, indicator: %s'%(name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')


###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   

    

######################################################################################################################
######################################################################################################################
def plot_1D_ASSG_4_all_ratings(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(number_of_raters*1.4)))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+1)*1.4])
    plt.yticks(list(range(1, number_of_raters+1)))
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('1D ASSG, indicator: %s'%(name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')

###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   



######################################################################################################################
######################################################################################################################

def plot_1D_ASSG_5_fit_ev(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(number_of_raters*1.4)))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    #ax.scatter(global_single_indicator[0], const[0], s=100, color='fuchsia', label='global rating', zorder=21)
#y-fit for expected values------------------------------------------------------------
    popty, pcovy = curve_fit(xconst,range(1, number_of_raters), ratermeanvaluelist(v))
    yfit_yconst=np.linspace(0, number_of_raters+1.5)
    xfit_yconst=popty[0]+0*yfit_yconst
    ax.plot(xfit_yconst, yfit_yconst, color='crimson', zorder=10)
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+1)*1.4])
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('1D ASSG, indicator: %s'%(name_indicator_plot), fontsize='22')
    ax.set_xlabel(
            ('Rating, $\overline{td}$=%.2f$\pm$%.2f,'
             '$\overline{tt}=%.2f$$\pm$%.2f')
                      %(round(meanlist(travel_distance_list), 2),
                        round(stdlist(travel_distance_list)/np.sqrt(number_of_raters), 2),
                        round(meanlist(travel_tendency_list), 2),
                        round(stdlist(travel_tendency_list)/np.sqrt(number_of_raters), 2)),
             fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')

###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   

    
######################################################################################################################
######################################################################################################################

def plot_1D_ASSG_6_mean_values_raters(path_data, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(number_of_raters*1.4)))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
#mean value of all ratings------------------------------------------------------------
    mean_ratings, error_ratings = ratings_mean_and_error(all_ratings, number_of_raters, number_of_ratings)
    const_mean_values=np.zeros(len(mean_ratings))+number_of_raters+1
    mean_ratings_reloc, mean_ratings_reloc_row = relocating(mean_ratings, const_mean_values)
    mean_values_plot_arrows= ax.plot(mean_ratings_reloc, mean_ratings_reloc_row,
                                 c='peachpuff', zorder=0)[0]
    mean_values_plot= ax.scatter(mean_ratings_reloc, mean_ratings_reloc_row, s=time_1D(mean_ratings,number_of_raters),
                                 c=df.z, cmap='RdPu', zorder=10)
    mean_values_plot= ax.scatter(mean_ratings_reloc[0], mean_ratings_reloc_row[0], s=time_1D(mean_ratings,number_of_raters)[0],
                                 c='hotpink', 
                                 label='mean values of raters'
                                 ,
                                 zorder=0)
#color for meanvalues between ratings
    color_lines = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
    points=np.array([mean_ratings_reloc, mean_ratings_reloc_row]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = mc.LineCollection(segments, cmap='RdPu', norm=norm)
    ax.add_collection(lc)
    lc.set_array(np.linspace(10, len(mean_ratings_reloc), len(mean_ratings_reloc)))
                
    add_arrow_hotpink(ax, mean_values_plot_arrows, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
    #EV
    popty, pcovy = curve_fit(xconst,range(1, number_of_raters), ratermeanvaluelist(v))
    ax.scatter(popty[0], number_of_raters+1, s=400, color='crimson',
                   label='$\overline{EV}$=%.2f'%(popty[0]), zorder=20)
    #start and end
    ax.scatter(mean_ratings_reloc[0],mean_ratings_reloc_row[0] ,s=50, color='limegreen', zorder=25)
    ax.scatter(mean_ratings_reloc[-1],mean_ratings_reloc_row[-1] ,s=20, color='gold', zorder=30)
    stdx=(np.linspace(meanlist(mean_ratings)-stdlist(mean_ratings),
                    meanlist(mean_ratings)+stdlist(mean_ratings), 50))
    #SD
    stdy=stdx*0+len(v)+1
    ax.plot(stdx, stdy, linewidth='3', color='orangered', 
                label='$\overline{SD}$=%.2f'%(mean_sd), zorder=15)
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+1)*1.4])
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('1D ASSG, indicator: %s'%(name_indicator_plot), fontsize='22')
    ax.set_xlabel(
            ('Rating, $\overline{td}$=%.2f,'
             '$\overline{tt}$=%.2f')
                      %(round(meanlist(travel_distance_list), 2),
                        round(meanlist(travel_tendency_list), 2)),
             fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')

###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   


In [ ]:
###------------------- 1D ASSG sucessive including global--------------------

def plot_1D_ASSG_including_global_2_single_rating_ev_sd(path_data, path_data_global, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    global_not_arranged=np.loadtxt(path_data_global, unpack=True)
    global_all=np.array(v_not_arranged).T
    global_single_indicator=global_all[q]
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(2.5)))
    grid_chessboard(6, int((2.5+3)*1.6-0.5))
    for j in range(len(v)-len(v)+1): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    ax.scatter(global_single_indicator[0], const[0], s=100, color='fuchsia', label='global rating', zorder=21)    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,2*1.4])
    plt.yticks(list(range(1, 2)))
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')
    ###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   

######################################################################################################################
######################################################################################################################

def plot_1D_ASSG_including_global_3_two_ratings(path_data, path_data_global, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    global_not_arranged=np.loadtxt(path_data_global, unpack=True)
    global_all=np.array(v_not_arranged).T
    global_single_indicator=global_all[q]
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(number_of_raters*1.4)))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)-len(v)+2): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    ax.scatter(global_single_indicator[0], const[0], s=100, color='fuchsia', label='global rating', zorder=21)    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,number_of_raters*1.4])
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')


###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   

    

######################################################################################################################
######################################################################################################################
def plot_1D_ASSG_including_global_4_all_ratings(path_data, path_data_global, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    global_not_arranged=np.loadtxt(path_data_global, unpack=True)
    global_all=np.array(v_not_arranged).T
    global_single_indicator=global_all[q]
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(number_of_raters*1.4)))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    ax.scatter(global_single_indicator[0], const[0], s=100, color='fuchsia', label='global rating', zorder=21)
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+1)*1.4])
    plt.yticks(list(range(1, number_of_raters+1)))
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel('Rating', fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')

###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   



######################################################################################################################
######################################################################################################################

def plot_1D_ASSG_including_global_5_fit_ev(path_data, path_data_global, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    global_not_arranged=np.loadtxt(path_data_global, unpack=True)
    global_all=np.array(v_not_arranged).T
    global_single_indicator=global_all[q]
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(number_of_raters*1.4)))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    #ax.scatter(global_single_indicator[0], const[0], s=100, color='fuchsia', label='global rating', zorder=21)
#y-fit for expected values------------------------------------------------------------
    popty, pcovy = curve_fit(xconst,range(1, number_of_raters), ratermeanvaluelist(v))
    yfit_yconst=np.linspace(0, number_of_raters+1.5)
    xfit_yconst=popty[0]+0*yfit_yconst
    ax.plot(xfit_yconst, yfit_yconst, color='crimson', zorder=10)
#y-fit for global values------------------------------------------------------------
    xfit_yconst=meanlist(global_single_indicator)+0*yfit_yconst
    ax.plot(xfit_yconst, yfit_yconst, color='fuchsia', zorder=10)
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+1)*1.4])
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel(
            ('Rating, $\overline{td}$=%.2f$\pm$%.2f,'
             '$\overline{tt}=%.2f$$\pm$%.2f')
                      %(round(meanlist(travel_distance_list), 2),
                        round(stdlist(travel_distance_list)/np.sqrt(number_of_raters), 2),
                        round(meanlist(travel_tendency_list), 2),
                        round(stdlist(travel_tendency_list)/np.sqrt(number_of_raters), 2)),
             fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')

###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   

    
######################################################################################################################
######################################################################################################################

def plot_1D_ASSG_including_global_6_mean_values_raters(path_data, path_data_global, path_fig, title_plot, name_indicator_plot):
    v_not_arranged=np.loadtxt(path_data, unpack=True)
    v=np.array(v_not_arranged).T
    global_not_arranged=np.loadtxt(path_data_global, unpack=True)
    global_all=np.array(v_not_arranged).T
    global_single_indicator=global_all[q]
    number_of_raters=len(v)
    number_of_ratings=len(v[0])
###---------------- set lists to fill with data -----------------------
    all_ratings=np.zeros(number_of_raters*number_of_ratings)
    meanvalue_sd=np.zeros(len(v))
    ev_list=np.zeros(len(v))
    travel_distance_list=np.zeros(len(v))
    travel_tendency_list=np.zeros(len(v))
###----------------------Plots-------------------------------------------
    fig, ax = plt.subplots(figsize=(3*(5.4-0.6),3*(number_of_raters*1.4)))
    grid_chessboard(6, int((number_of_raters+3)*1.6-0.5))
    for j in range(len(v)): #slope of rater
#standard differences
                const=np.zeros(len(v[j]))+1+j
                stdx=(np.linspace(meanlist(v[j])-stdlist(v[j]),
                    meanlist(v[j])+stdlist(v[j]), 50))
                stdy=stdx*0+j+1
#plots of raters
                reloc_rater, reloc_row =relocating(v[j], const)
                df = pd.DataFrame({'z': (np.linspace(0, len(v[j]), len(v[j])))})
                
                line=plt.plot(reloc_rater, reloc_row, c='skyblue', zorder=-5)[0]
                add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
                line1= (ax.scatter(reloc_rater, reloc_row,s=time_1D(v[j],number_of_raters),
                                  c=df.z, cmap='Blues', zorder=10))
                ax.plot(stdx, stdy, linewidth='3', color='orangered', zorder=15)
                ax.scatter(meanlist(v[j]),j+1 ,s=2000/number_of_raters, color='crimson', zorder=20)
                ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', zorder=25)
                ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=40, color='gold', zorder=30)
                
                #color for each line between ratings
                color_lines = (np.linspace(0, len(reloc_rater), len(reloc_rater)))
                norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
                points=np.array([reloc_rater, reloc_row]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
                ax.add_collection(lc)
                lc.set_array(np.linspace(10, len(reloc_rater), len(reloc_rater)))
                
                #data
                meanvalue_sd[j] += stdlist(v[j])
                ev_list[j]+= meanlist(v[j])
                travel_distance_list[j] += 2*traveldistance(v[j], const)
                travel_tendency_list[j] += 2*traveltendency(v[j], const)
                for k in range(len(v[j])):
                    position_rating=k+j*len(v[j])
                    all_ratings[position_rating]=+(v[j][k])
                
#plots for labels
    mean_sd=meanlist(meanvalue_sd)
    const=np.zeros(len(v[0]))+1
    reloc_rater, reloc_row =relocating(v[0], const)
    line0= ax.scatter(reloc_rater[2], reloc_row[2], s=time_1D(v[j],number_of_raters)[2],
                      color='skyblue', label='ratings of each rater', zorder=0)
    line= ax.scatter(reloc_rater, reloc_row, s=time_1D(v[j],number_of_raters),
                     c=df.z, cmap='Blues', zorder=5)
        
    (ax.plot(np.linspace(meanlist(v[0])-stdlist(v[0]),
                            meanlist(v[0])+stdlist(v[0]), 50),
                            np.linspace(meanlist(v[0])-
                                        stdlist(v[0]),
                            (meanlist(v[0])+
                             stdlist(v[0])), 50)*0+1,
                            linewidth='3', color='orangered',
                            label='SD of rater', zorder=10))
    ax.scatter(meanlist(v[0]),1 ,s=100, color='crimson',
                   label='EV of rater', zorder=15)
    ax.scatter(reloc_rater[0],reloc_row[0] ,s=50, color='limegreen', label='first rating', zorder=20)
    ax.scatter(reloc_rater[-1],reloc_row[-1] ,s=20, color='gold', label='last rating', zorder=25)
    ax.scatter(global_single_indicator[0], const[0], s=100, color='fuchsia', label='global rating', zorder=21)
#y-fit for expected values------------------------------------------------------------
    popty, pcovy = curve_fit(xconst,range(1, number_of_raters), ratermeanvaluelist(v))
    yfit_yconst=np.linspace(0, number_of_raters+1.5)
    xfit_yconst=popty[0]+0*yfit_yconst
    ax.plot(xfit_yconst, yfit_yconst, color='crimson', zorder=10)
#y-fit for global values------------------------------------------------------------
    xfit_yconst=meanlist(global_single_indicator)+0*yfit_yconst
    ax.plot(xfit_yconst, yfit_yconst, color='fuchsia', zorder=10)
#mean value of all ratings------------------------------------------------------------
    mean_ratings, error_ratings = ratings_mean_and_error(all_ratings, number_of_raters, number_of_ratings)
    const_mean_values=np.zeros(len(mean_ratings))+number_of_raters+1
    mean_ratings_reloc, mean_ratings_reloc_row = relocating(mean_ratings, const_mean_values)
    mean_values_plot_arrows= ax.plot(mean_ratings_reloc, mean_ratings_reloc_row,
                                 c='peachpuff', zorder=0)[0]
    mean_values_plot= ax.scatter(mean_ratings_reloc, mean_ratings_reloc_row, s=time_1D(mean_ratings,number_of_raters),
                                 c=df.z, cmap='OrRd', zorder=10)
    mean_values_plot= ax.scatter(mean_ratings_reloc[0], mean_ratings_reloc_row[0], s=time_1D(mean_ratings,number_of_raters)[0],
                                 c='peachpuff', 
                                 label='mean values of raters \n$\overline{\Delta rating}=$%.2f$\pm$%.2f'
                                 %(meanlist(error_ratings), stdlist(error_ratings)),
                                 zorder=0)
#color for meanvalues between ratings
    color_lines = (np.linspace(0, len(mean_ratings_reloc), len(mean_ratings_reloc)))
    norm = Normalize(vmin=color_lines.min(), vmax=color_lines.max())
                
    points=np.array([mean_ratings_reloc, mean_ratings_reloc_row]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = mc.LineCollection(segments, cmap='OrRd', norm=norm)
    ax.add_collection(lc)
    lc.set_array(np.linspace(10, len(mean_ratings_reloc), len(mean_ratings_reloc)))
                
    add_arrow_peachpuff(ax, mean_values_plot_arrows, arrow_locs=np.linspace(0, 1, 30),
                          arrowstyle='->', arrowsize=4)
    #EV
    ax.scatter(popty[0], number_of_raters+1, s=400, color='purple',
                   label='$\overline{EV}$=%.2f$\pm$%.2f'%(popty[0], stdlist(ev_list)/np.sqrt(number_of_raters)), zorder=15)
    #start and end
    ax.scatter(mean_ratings_reloc[0],mean_ratings_reloc_row[0] ,s=50, color='limegreen', zorder=25)
    ax.scatter(mean_ratings_reloc[-1],mean_ratings_reloc_row[-1] ,s=20, color='gold', zorder=30)
    stdx=(np.linspace(meanlist(mean_ratings)-stdlist(mean_ratings),
                    meanlist(mean_ratings)+stdlist(mean_ratings), 50))
    #SD
    stdy=stdx*0+len(v)+1
    ax.plot(stdx, stdy, linewidth='3', color='purple', 
                label='$\overline{SD}$=%.2f$\pm$%.2f'%(mean_sd, stdlist(meanvalue_sd)/np.sqrt(number_of_raters)), zorder=20)
    #global    
    ax.scatter(meanlist(global_single_indicator), const_mean_values[0], s=400, color='fuchsia',
               label='global ratings \n$|\overline{global}-\overline{EV}|$=%.2f'
               %(np.sqrt((popty[0]-meanlist(global_single_indicator))**2)),
               zorder=21)
    #limit of Axes
    ax.set_xlim([0.6,5+0.4])
    ax.set_ylim([0.5,(number_of_raters+1)*1.4])
    plt.yticks(list(range(1, number_of_raters+1)))
    #axes and title
    ax.set_title('%s, indicator: %s'%(title_plot, name_indicator_plot), fontsize='22')
    ax.set_xlabel(
            ('Rating, $\overline{td}$=%.2f$\pm$%.2f,'
             '$\overline{tt}=%.2f$$\pm$%.2f')
                      %(round(meanlist(travel_distance_list), 2),
                        round(stdlist(travel_distance_list)/np.sqrt(number_of_raters), 2),
                        round(meanlist(travel_tendency_list), 2),
                        round(stdlist(travel_tendency_list)/np.sqrt(number_of_raters), 2)),
             fontsize='28')
    #ax.set_ylabel('Rater, ICC=%.2f'
    #                  %(icc(all_ratings, mean_ratings, number_of_raters, number_of_ratings)), fontsize='24')
    ax.set_ylabel('Rater', fontsize='24')

###-------------------------------------Cosmetics-----------------------
#legend
    ax.legend(ncol=2, fontsize='20').set_zorder(100)
###-------------------------------------Saving---------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()   
